In [1]:
pip install dash plotly pandas


  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.1.0-cp312-cp312-win_amd64.whl.metadata (59 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/7.5 MB 1.7 MB/s eta 0:00:05
   ---------------------------------------- 0.1/7.5 MB 653.6 kB/s eta 0:00:12
    --------------------------------------- 0.2/7.5 MB 1.1 MB/s eta 0:00:07
   - -------------------------------------- 0.2/7.5 MB 958.4 kB/s eta 0:00:08
   - -------------------------------------- 0.3/7.5 MB 923.9 kB/s eta 0:00:08
   - -------------------------------------- 0.3/7.5 MB 1.0 MB/s eta 0:00:07
   - -------------------------------------- 0.3/7.5 MB 1.0 MB/s eta 0:00:07
   -- ------------------------------------- 0.4/7.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
df=pd.read_csv(r'C:\Users\HP\Desktop\merged_dataset.csv')

In [4]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')


In [12]:
import pandas as pd
import plotly.graph_objects as go
from dash import dcc, html, Dash, Input, Output

# Load and preprocess your dataset

df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
df['Month'] = df['Timestamp'].dt.month

# Create the Dash app
app = Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Solar Radiation and Environmental Data Dashboard"),
    
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=df['Timestamp'].min(),
        end_date=df['Timestamp'].max(),
        display_format='YYYY-MM-DD',
    ),

    dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'GHI', 'value': 'GHI'},
            {'label': 'DNI', 'value': 'DNI'},
            {'label': 'DHI', 'value': 'DHI'},
            {'label': 'Temperature A', 'value': 'TModA'},
            {'label': 'Temperature B', 'value': 'TModB'},
            {'label': 'Wind Speed', 'value': 'WS'},
            {'label': 'Wind Gust', 'value': 'WSgust'},
            {'label': 'Relative Humidity', 'value': 'RH'}
        ],
        value=['GHI', 'DNI', 'DHI'],  # Default selected values
        multi=True
    ),

    dcc.RadioItems(
        id='theme-toggle',
        options=[
            {'label': 'Light Mode', 'value': 'plotly_white'},
            {'label': 'Dark Mode', 'value': 'plotly_dark'}
        ],
        value='plotly_white'
    ),

    dcc.Graph(id='histograms')
])

# Callback for histograms
@app.callback(
    Output('histograms', 'figure'),
    [Input('variable-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('theme-toggle', 'value')]
)
def update_histograms(variables, start_date, end_date, theme):
    filtered_df = df[(df['Timestamp'] >= start_date) & (df['Timestamp'] <= end_date)]
    
    fig = go.Figure()

    for var in variables:
        fig.add_trace(go.Histogram(x=filtered_df[var], name=var, opacity=0.6))

    # Update layout
    fig.update_layout(
        barmode='overlay',
        template=theme,
        title='Histograms of Selected Variables',
        xaxis_title='Value',
        yaxis_title='Frequency',
        legend_title='Variables',
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
